In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
# !pip install ipywidgets

In [3]:
# !pip install -U git+https://github.com/albumentations-team/albumentations

In [4]:
# 처음 한번만 받으면 됨
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [5]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 32
cfg.data.workers_per_gpu = 0

cfg.seed=2020
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash_std'
cfg.load_from = 'faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

cfg.model.roi_head.bbox_head.num_classes = 11
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.runner.max_epochs = 13

cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[20, 30, 40])
cfg.log_config = dict(  # config to register logger hook
    interval=40,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
])

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [6]:
model = build_detector(cfg.model)

2021-05-13 06:15:27,811 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-13 06:15:27,812 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 06:15:28,054 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.83s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-13 06:15:35,975 - mmdet - INFO - load checkpoint from faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2021-05-13 06:15:35,976 - mmdet - INFO - Use load_from_local loader
2021-05-13 06:15:36,133 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
2021-05-13 06:15:36,138 - mmdet - INFO - Start running, 

Done (t=0.88s)
creating index...
index created!


2021-05-13 06:17:15,108 - mmdet - INFO - Epoch [1][40/82]	lr: 1.578e-03, eta: 0:42:14, time: 2.470, data_time: 0.395, memory: 13295, loss_rpn_cls: 0.1316, loss_rpn_bbox: 0.0534, loss_cls: 1.1077, acc: 73.6685, loss_bbox: 0.4820, loss: 1.7747, grad_norm: 5.4359
2021-05-13 06:18:50,274 - mmdet - INFO - Epoch [1][80/82]	lr: 3.177e-03, eta: 0:39:50, time: 2.379, data_time: 0.340, memory: 13295, loss_rpn_cls: 0.0777, loss_rpn_bbox: 0.0478, loss_cls: 0.4770, acc: 87.2243, loss_bbox: 0.4756, loss: 1.0781, grad_norm: 1.2095
2021-05-13 06:18:55,146 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 18.0 task/s, elapsed: 36s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.60s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.57s).
Accumulating evaluation results...
DONE (t=1.26s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.046
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.089
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.042
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.013
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.047
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.077
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.104
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.104
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.104
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-13 06:21:21,233 - mmdet - INFO - Epoch [2][40/82]	lr: 4.855e-03, eta: 0:37:41, time: 2.458, data_time: 0.404, memory: 13295, loss_rpn_cls: 0.0738, loss_rpn_bbox: 0.0493, loss_cls: 0.4000, acc: 89.1681, loss_bbox: 0.3731, loss: 0.8963, grad_norm: 1.3509
2021-05-13 06:22:56,247 - mmdet - INFO - Epoch [2][80/82]	lr: 6.454e-03, eta: 0:36:01, time: 2.375, data_time: 0.334, memory: 13295, loss_rpn_cls: 0.0656, loss_rpn_bbox: 0.0460, loss_cls: 0.3676, acc: 89.7015, loss_bbox: 0.3186, loss: 0.7978, grad_norm: 1.3896
2021-05-13 06:23:01,015 - mmdet - INFO - Saving checkpoint at 2 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 17.9 task/s, elapsed: 37s, ETA:     0s

2021-05-13 06:23:38,853 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.54s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.36s).
Accumulating evaluation results...
DONE (t=1.02s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.106
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.184
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.104
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.023
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.116
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.197
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.244
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.244
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.244
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]